In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 596.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 102.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.5 MB/s eta 0:00:00


In [5]:
import os
import h5py
import numpy as np
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
import tensorflow as tf

In [26]:
import numpy as np
import h5py
import glob
import os
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# --- 1. SETUP PATHS AND CONSTANTS ---
print("--- Step 1: Setting up paths and constants ---")
# Paths from your provided code, with an added path for the INTRA dataset
BASE_PATH_CROSS = '/content/drive/MyDrive/DeepLearning Project/Final Project data/Cross'
BASE_PATH_INTRA = '/content/drive/MyDrive/DeepLearning Project/Final Project data/Intra' #<-- ADD THIS PATH

# Define data directories
CROSS_TRAIN_DIR = os.path.join(BASE_PATH_CROSS, 'train/')
TEST1_DIR = os.path.join(BASE_PATH_CROSS, 'test1/')
TEST2_DIR = os.path.join(BASE_PATH_CROSS, 'test2/')
TEST3_DIR = os.path.join(BASE_PATH_CROSS, 'test3/')
INTRA_TRAIN_DIR = os.path.join(BASE_PATH_INTRA, 'train/') #<-- ADD THIS PATH

# Constants from the project description
N_CHANNELS = 248
TASKS = ['rest', 'task_motor', 'task_story_math', 'task_working_memory']
task_to_label = {task: i for i, task in enumerate(TASKS)}


# --- 2. DATA LOADING FUNCTIONS (with robust checks) ---
def load_data(file_paths):
    """Loads MEG data and labels from a list of .h5 file paths."""
    data = []
    labels = []
    for file_path in file_paths:
        filename = file_path.split('/')[-1]
        if 'rest' in filename:
            labels.append(task_to_label['rest'])
        elif 'motor' in filename:
            labels.append(task_to_label['task_motor'])
        elif 'story' in filename or 'math' in filename:
             labels.append(task_to_label['task_story_math'])
        elif 'working' in filename or 'memory' in filename:
            labels.append(task_to_label['task_working_memory'])
        else:
            print(f"Warning: Could not determine task for file: {filename}")
            continue

        with h5py.File(file_path, 'r') as f:
            dataset_name = list(f.keys())[0]
            matrix = f[dataset_name][()]
            data.append(matrix)

    return np.array(data), np.array(labels)

def get_file_paths(directory):
    """Safely gets file paths and throws an error if the directory is empty."""
    files = glob.glob(f"{directory}/*.h5")
    if not files:
        print(f"Couldn't mount the data")
        raise FileNotFoundError(f"No files found in {directory}")
    return files


# Combining datasets
print("\n--- Step 2: Loading and Combining Data ---")
try:
    cross_train_files = get_file_paths(CROSS_TRAIN_DIR)
    intra_train_files = get_file_paths(INTRA_TRAIN_DIR)
    test1_files = get_file_paths(TEST1_DIR)
    test2_files = get_file_paths(TEST2_DIR)
    test3_files = get_file_paths(TEST3_DIR)

    X_train_cross, y_train_cross = load_data(cross_train_files)
    X_train_intra, y_train_intra = load_data(intra_train_files)
    X_test1, y_test1 = load_data(test1_files)
    X_test2, y_test2 = load_data(test2_files)
    X_test3, y_test3 = load_data(test3_files)

    # combine cross and intra training sets
    print(f"Combining {X_train_cross.shape[0]} Cross-subject samples with {X_train_intra.shape[0]} Intra-subject samples.")
    X_train = np.concatenate((X_train_cross, X_train_intra), axis=0)
    y_train = np.concatenate((y_train_cross, y_train_intra), axis=0)
    print(f"New combined training set shape: {X_train.shape}")

except FileNotFoundError:
    print("\nData loading failed. Please fix the directory paths and try again.")
else:
    print("\nDownsampling")
    DOWNSAMPLE_FACTOR = 10
    X_train_ds = X_train[:, :, ::DOWNSAMPLE_FACTOR]
    X_test1_ds = X_test1[:, :, ::DOWNSAMPLE_FACTOR]
    X_test2_ds = X_test2[:, :, ::DOWNSAMPLE_FACTOR]
    X_test3_ds = X_test3[:, :, ::DOWNSAMPLE_FACTOR]
    N_TIMESTEPS_DS = X_train_ds.shape[2]
    print(f"Data downsampled by factor of {DOWNSAMPLE_FACTOR}. New time steps: {N_TIMESTEPS_DS}")

    # --- 5. NORMALIZATION ---
    def normalize_data(data):
        """Applies time-wise Z-score normalization."""
        n_samples, n_channels, n_timesteps = data.shape
        reshaped_data = data.reshape(n_samples * n_channels, n_timesteps)
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(reshaped_data)
        return scaled_data.reshape(n_samples, n_channels, n_timesteps)

    print("\n--- Step 4: Normalizing Data ---")
    X_train_norm = normalize_data(X_train_ds)
    X_test1_norm = normalize_data(X_test1_ds)
    X_test2_norm = normalize_data(X_test2_ds)
    X_test3_norm = normalize_data(X_test3_ds)
    print("All datasets normalized.")

    # --- 6. DATA AUGMENTATION ---
    def add_noise(data, noise_factor=0.05):
        noise = np.random.normal(loc=0.0, scale=noise_factor, size=data.shape)
        return data + noise

    def scale_amplitude(data, scale_factor_range=(0.9, 1.1)):
        scaler = np.random.uniform(low=scale_factor_range[0], high=scale_factor_range[1])
        return data * scaler

    print("\n--- Step 5: Augmenting Training Data ---")
    X_train_augmented_list = []
    y_train_augmented_list = []

    for i in range(len(X_train_norm)):
        original_sample = X_train_norm[i]
        original_label = y_train[i]
        X_train_augmented_list.append(original_sample)
        y_train_augmented_list.append(original_label)

        augmented_sample = add_noise(original_sample)
        augmented_sample = scale_amplitude(augmented_sample)
        X_train_augmented_list.append(augmented_sample)
        y_train_augmented_list.append(original_label)

    X_train_augmented = np.array(X_train_augmented_list)
    y_train_augmented = np.array(y_train_augmented_list)
    print(f"Augmentation complete. New training set shape: {X_train_augmented.shape}")

    # --- 7. FINAL RESHAPING FOR MODELS ---
    print("\n--- Step 6: Finalizing Data Shapes ---")

    # Shape for 1D models (CNN, LSTM): (samples, timesteps, channels)
    X_train_final_1D = np.transpose(X_train_augmented, (0, 2, 1))
    X_test1_final_1D = np.transpose(X_test1_norm, (0, 2, 1))
    X_test2_final_1D = np.transpose(X_test2_norm, (0, 2, 1))
    X_test3_final_1D = np.transpose(X_test3_norm, (0, 2, 1))
    print(f"Shape for 1D models (e.g., CNN): {X_train_final_1D.shape}")

    # Shape for 2D models (EEGNet, AA-CascadeNet): (samples, channels, timesteps, 1)
    X_train_final_2D = X_train_augmented[..., np.newaxis]
    X_test1_final_2D = X_test1_norm[..., np.newaxis]
    X_test2_final_2D = X_test2_norm[..., np.newaxis]
    X_test3_final_2D = X_test3_norm[..., np.newaxis]
    print(f"Shape for 2D models (e.g., EEGNet): {X_train_final_2D.shape}")

    print("\nPreprocessing complete and all datasets are ready!")

--- Step 1: Setting up paths and constants ---

--- Step 2: Loading and Combining Data ---
Combining 64 Cross-subject samples with 32 Intra-subject samples.
New combined training set shape: (96, 248, 35624)

--- Step 3: Downsampling Data ---
Data downsampled by factor of 10. New time steps: 3563

--- Step 4: Normalizing Data ---
All datasets normalized.

--- Step 5: Augmenting Training Data ---
Augmentation complete. New training set shape: (192, 248, 3563)

--- Step 6: Finalizing Data Shapes ---
Shape for 1D models (e.g., CNN): (192, 3563, 248)
Shape for 2D models (e.g., EEGNet): (192, 248, 3563, 1)

Preprocessing complete and all datasets are ready!


In [8]:
# Code Block 3: 1D CNN Model Architecture
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),

        #1st convolutional block
        Conv1D(filters=64, kernel_size=10, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=4),

        #2nd convolutional block
        Conv1D(filters=128, kernel_size=10, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=4),

        # 3rd convolutional block
        Conv1D(filters=256, kernel_size=10, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=4),

        # Flatten the features and feed to dense layers
        Flatten(),

        # dense layers for classification
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    return model

# define model parameters
INPUT_SHAPE = (N_TIMESTEPS_DS, N_CHANNELS)
NUM_CLASSES = len(TASKS)

cnn_model = build_cnn_model(INPUT_SHAPE, NUM_CLASSES)
cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', # Use sparse CE because our labels are integers
    metrics=['accuracy']
)
cnn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 3563, 64)       │       158,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 3563, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 890, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 890, 128)       │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 890, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 222, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 222, 256)       │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 222, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 55, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 14080)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,802,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,373,444 (9.05 MB)

 Trainable params: 2,372,548 (9.05 MB)

 Non-trainable params: 896 (3.50 KB)

In [27]:
#CNN Training and Evaluation
history_cnn = cnn_model.fit(
    X_train_final_1D,
    y_train_augmented,
    epochs=20,
    batch_size=16,      # smaller batch size for better generalization
    validation_split=0.2 # 20% of training data for validation
)

# Evaluate on test Sets
loss1_cnn, acc1_cnn = cnn_model.evaluate(X_test1_final_1D, y_test1, verbose=0)
print(f"accuracy on test set 1: {acc1_cnn * 100:.2f}%")
loss2_cnn, acc2_cnn = cnn_model.evaluate(X_test2_final_1D, y_test2, verbose=0)
print(f"accuracy on test set 2: {acc2_cnn * 100:.2f}%")
loss3_cnn, acc3_cnn = cnn_model.evaluate(X_test3_final_1D, y_test3, verbose=0)
print(f"accuracy on test set 3: {acc3_cnn * 100:.2f}%")

Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.9286 - loss: 2.3184 - val_accuracy: 0.9231 - val_loss: 0.1595
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.8279 - loss: 4.4062 - val_accuracy: 0.8974 - val_loss: 0.2091
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.8686 - loss: 1.7767 - val_accuracy: 1.0000 - val_loss: 1.1243e-04
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.9076 - loss: 1.3145 - val_accuracy: 1.0000 - val_loss: 2.1397e-08
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.9147 - loss: 0.7613 - val_accuracy: 1.0000 - val_loss: 2.4453e-08
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.9171 - loss: 1.2744 - val_accuracy: 1.0000 - val_loss: 9.1699e-09
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.9673 - loss: 0.1408 - val_accuracy: 1.0000 - val_loss: 1.3877e-06
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.9701 - loss: 0.1133 - val_accuracy

In [28]:
#Hybrid CNN-LSTM Model Architecture
from tensorflow.keras.layers import LSTM

def build_cnn_lstm_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),

        # Convolutional layers to extract features
        Conv1D(filters=64, kernel_size=10, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=4),

        Conv1D(filters=128, kernel_size=10, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=4),

        # LSTM layer to model temporal sequences of the extracted features
        LSTM(128, return_sequences=False), # return_sequences=False  it's the last recurrent layer

        # Dense layers for classification
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    return model

cnn_lstm_model = build_cnn_lstm_model(INPUT_SHAPE, NUM_CLASSES)

cnn_lstm_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

cnn_lstm_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_5 (Conv1D)               │ (None, 3563, 64)       │       158,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 3563, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 890, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, 890, 128)       │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 890, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_6 (MaxPooling1D)  │ (None, 222, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 390,212 (1.49 MB)

 Trainable params: 389,828 (1.49 MB)

 Non-trainable params: 384 (1.50 KB)

In [29]:
#CNN-LSTM Training and Evaluation
print("Starting CNN-LSTM model training...")
history_cnn_lstm = cnn_lstm_model.fit(
    X_train_final_1D,
    y_train_augmented,
    epochs=20,
    batch_size=16,
    validation_split=0.2
)

# Evaluate on test sets
loss1_hybrid, acc1_hybrid = cnn_lstm_model.evaluate(X_test1_final_1D, y_test1, verbose=0)
print(f"Hybrid Model Accuracy on Test Set 1: {acc1_hybrid * 100:.2f}%")
loss2_hybrid, acc2_hybrid = cnn_lstm_model.evaluate(X_test2_final_1D, y_test2, verbose=0)
print(f"Hybrid Model Accuracy on Test Set 2: {acc2_hybrid * 100:.2f}%")
loss3_hybrid, acc3_hybrid = cnn_lstm_model.evaluate(X_test3_final_1D, y_test3, verbose=0)
print(f"Hybrid Model Accuracy on Test Set 3: {acc3_hybrid * 100:.2f}%")

Starting CNN-LSTM model training...
Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.5054 - loss: 1.1770 - val_accuracy: 0.5385 - val_loss: 1.1825
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.9334 - loss: 0.3679 - val_accuracy: 0.5385 - val_loss: 0.9355
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.9551 - loss: 0.1942 - val_accuracy: 0.7949 - val_loss: 0.5224
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.9768 - loss: 0.0876 - val_accuracy: 0.8462 - val_loss: 0.4135
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 1.0000 - loss: 0.0104 - val_accuracy: 0.8718 - val_loss: 0.2155
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 1.0000 - loss: 0.0135 - val_accuracy: 0.9231 - val_loss: 0.1026
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.9946 - loss: 0.0221 - val_accuracy: 0.9487 - val_loss: 0.1160
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 1.0000 - loss: 0.009

In [18]:
#EEGNet model
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, Dropout, BatchNormalization, Activation, AveragePooling2D, Flatten, DepthwiseConv2D, SeparableConv2D
from tensorflow.keras.callbacks import EarlyStopping

def build_eegnet_model(num_classes, channels, timesteps, dropout_rate=0.5):
    input_layer = Input(shape=(channels, timesteps, 1))

    # temporal convolution
    block1 = Conv2D(16, (1, 64), padding='same', use_bias=False)(input_layer)
    block1 = BatchNormalization()(block1)

    #Depthwise spatial convolution
    block1 = DepthwiseConv2D((channels, 1), use_bias=False, depth_multiplier=2, depthwise_constraint=tf.keras.constraints.max_norm(1.))(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation('elu')(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = Dropout(dropout_rate)(block1)

    # separable convolution
    block2 = SeparableConv2D(32, (1, 16), use_bias=False, padding='same')(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((1, 8))(block2)
    block2 = Dropout(dropout_rate)(block2)

    # classification head
    flatten_layer = Flatten()(block2)
    dense_layer = Dense(num_classes, kernel_constraint=tf.keras.constraints.max_norm(0.25))(flatten_layer)
    output_layer = Activation('softmax')(dense_layer)

    return Model(inputs=input_layer, outputs=output_layer)

# Reshape data
X_train_eegnet = X_train_norm[..., np.newaxis]
X_test1_eegnet = X_test1_norm[..., np.newaxis]
X_test2_eegnet = X_test2_norm[..., np.newaxis]
X_test3_eegnet = X_test3_norm[..., np.newaxis]

print(f"Shape of data for EEGNet: {X_train_eegnet.shape}")

eegnet_model = build_eegnet_model(
    num_classes=NUM_CLASSES,
    channels=N_CHANNELS,
    timesteps=N_TIMESTEPS_DS
)

eegnet_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

eegnet_model.summary()

# Define the earlystopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

print("\nStarting EEGNet model training with Early Stopping...")
X_train_augmented_eegnet = X_train_augmented[..., np.newaxis]
X_train_augmented_eegnet = np.transpose(X_train_augmented_eegnet, (0, 2, 1, 3))



Shape of data for EEGNet: (64, 248, 3563, 1)


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 248, 3563, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 248, 3563, 16)  │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 248, 3563, 16)  │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_1              │ (None, 1, 3563, 32)    │         7,936 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 1, 3563, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 1, 3563, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_2             │ (None, 1, 890, 32)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 1, 890, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_1              │ (None, 1, 890, 32)     │         1,536 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 1, 890, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 1, 890, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_3             │ (None, 1, 111, 32)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 1, 111, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 3552)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │        14,212 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 4)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,028 (97.77 KB)

 Trainable params: 24,868 (97.14 KB)

 Non-trainable params: 160 (640.00 B)


Starting EEGNet model training with Early Stopping...


In [30]:
# 3. Train the model with the augmented data
history_eegnet = eegnet_model.fit(
    X_train_final_2D,  # Use the newly shaped augmented data
    y_train_augmented,         # Use the corresponding augmented labels
    epochs=50,
    batch_size=16,
    validation_split=0.2,      # The split is now on the larger augmented dataset
    callbacks=[early_stopping]
)

print("\nEvaluating EEGNet model on test sets")
loss1_eegnet, acc1_eegnet = eegnet_model.evaluate(X_test1_final_2D, y_test1, verbose=0)
print(f"EEGNet Model Accuracy on Test Set 1: {acc1_eegnet * 100:.2f}%")

loss2_eegnet, acc2_eegnet = eegnet_model.evaluate(X_test2_final_2D, y_test2, verbose=0)
print(f"EEGNet Model Accuracy on Test Set 2: {acc2_eegnet * 100:.2f}%")

loss3_eegnet, acc3_eegnet = eegnet_model.evaluate(X_test3_final_2D, y_test3, verbose=0)
print(f"EEGNet Model Accuracy on Test Set 3: {acc3_eegnet * 100:.2f}%")

Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 221s 22s/step - accuracy: 0.9279 - loss: 0.2551 - val_accuracy: 1.0000 - val_loss: 0.0418
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 219s 22s/step - accuracy: 0.9455 - loss: 0.1420 - val_accuracy: 1.0000 - val_loss: 0.0347
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 220s 22s/step - accuracy: 0.9913 - loss: 0.0405 - val_accuracy: 1.0000 - val_loss: 0.0054
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 218s 22s/step - accuracy: 1.0000 - loss: 0.0077 - val_accuracy: 1.0000 - val_loss: 0.0033
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 223s 22s/step - accuracy: 1.0000 - loss: 0.0089 - val_accuracy: 1.0000 - val_loss: 0.0042
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 221s 22s/step - accuracy: 1.0000 - loss: 0.0060 - val_accuracy: 1.0000 - val_loss: 0.0013
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 219s 22s/step - accuracy: 1.0000 - loss: 0.0029 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 8/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 221s 22s/step - accuracy: 1.0000 - loss: 0.0037 - val_accuracy: 1.

In [ ]:
# Final Code Block: Adapted AA-CascadeNet Model
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, Dense, Dropout, BatchNormalization,
                                     Activation, AveragePooling2D, Flatten, Multiply,
                                     Permute, Reshape, Softmax, Add)
from tensorflow.keras.callbacks import EarlyStopping

def AALayer(x, reduction=8):
    """
    Creates the Attention-Adjacency (AA) Layer with corrected shapes.
    This layer learns the dynamic relationships between MEG channels.
    """
    # Get the number of channels from the input shape
    in_channels = x.shape[1] # This should be 248

    # Global Average Pooling to get a summary of each channel's features
    # Input shape: (batch, channels, timesteps, filters)
    # Output shape: (batch, 1, 1, filters)
    y = tf.keras.layers.GlobalAveragePooling2D()(x)

    # Reshape for the Dense layers
    # Output shape: (batch, filters)
    y = Reshape((1, 1, y.shape[-1]))(y)

    # Squeeze-and-Excitation block to compute channel attentions
    # This learns which channels are most important
    # Output shape: (batch, filters // reduction) -> (batch, filters)
    y = Dense(in_channels // reduction, activation='relu')(y)
    y = Dense(in_channels, activation='sigmoid')(y)

    # Reshape the attention weights to be broadcastable
    # Output shape: (batch, channels, 1, 1)
    y = Reshape((in_channels, 1, 1))(y)

    # Multiply the original input by the learned attention weights
    # The shape (batch, channels, 1, 1) will broadcast over the timesteps and filters of x
    return Multiply()([x, y])

# --- Build the Full AA-CascadeNet Architecture ---
def build_adapted_aa_cascade_net(num_classes, channels, timesteps):
    """
    Builds the AA-CascadeNet model, adapted for our data format.
    """
    # Input shape for this model is (channels, timesteps, 1)
    input_layer = Input(shape=(channels, timesteps, 1))

    # --- Cascade-Block 1 ---
    aa1 = AALayer(input_layer)
    conv1 = Conv2D(filters=16, kernel_size=(1, 32), padding='same')(aa1)
    # Add a residual connection
    res1 = Conv2D(filters=16, kernel_size=1, padding='same')(input_layer)
    x = Add()([conv1, res1])
    x = BatchNormalization()(x)
    x = Activation('elu')(x)
    x = AveragePooling2D(pool_size=(1, 4))(x)
    x = Dropout(0.3)(x)

    # --- Cascade-Block 2 ---
    aa2 = AALayer(x)
    conv2 = Conv2D(filters=32, kernel_size=(1, 16), padding='same')(aa2)
    # Add a residual connection
    res2 = Conv2D(filters=32, kernel_size=1, padding='same')(x)
    x = Add()([conv2, res2])
    x = BatchNormalization()(x)
    x = Activation('elu')(x)
    x = AveragePooling2D(pool_size=(1, 4))(x)
    x = Dropout(0.3)(x)

    # --- Classification Head ---
    flatten_layer = Flatten()(x)
    dense_layer = Dense(128, activation='elu')(flatten_layer)
    dense_layer = Dropout(0.5)(dense_layer)
    output_layer = Dense(num_classes, activation='softmax')(dense_layer)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# --- Prepare Data, Build, and Train the Model ---

# NOTE: This model uses the same data shape as EEGNet.
# Make sure the variables from your previous cells are available:
# X_train_augmented_eegnet, y_train_augmented, X_test1_eegnet, y_test1, etc.

print("Building the Adapted AA-CascadeNet model...")
final_aa_model = build_adapted_aa_cascade_net(
    num_classes=NUM_CLASSES,
    channels=N_CHANNELS,
    timesteps=N_TIMESTEPS_DS
)

final_aa_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

final_aa_model.summary()

# Define EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10, # More patience for this complex model
    restore_best_weights=True
)

print("\nStarting Adapted AA-CascadeNet model training...")
history_final_aa = final_aa_model.fit(
    X_train_final_2D,
    y_train_augmented,
    epochs=100,
    batch_size=16,
    validation_split=0.2,
    callbacks=[early_stopping]
)

# --- Evaluate the Final Model ---
print("\nEvaluating Adapted AA-CascadeNet model on test sets...")
loss1, acc1 = final_aa_model.evaluate(X_test1_final_2D, y_test1, verbose=0)
print(f"Final Model Accuracy on Test Set 1: {acc1 * 100:.2f}%")

loss2, acc2 = final_aa_model.evaluate(X_test2_final_2D, y_test2, verbose=0)
print(f"Final Model Accuracy on Test Set 2: {acc2 * 100:.2f}%")

loss3, acc3 = final_aa_model.evaluate(X_test3_final_2D, y_test3, verbose=0)
print(f"Final Model Accuracy on Test Set 3: {acc3 * 100:.2f}%")

Building the Adapted AA-CascadeNet model...


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 248, 3563, │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1)         │          0 │ input_layer_8[0]… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_4 (Reshape) │ (None, 1, 1, 1)   │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_18 (Dense)    │ (None, 1, 1, 31)  │         62 │ reshape_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_19 (Dense)    │ (None, 1, 1, 248) │      7,936 │ dense_18[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_5 (Reshape) │ (None, 248, 1, 1) │          0 │ dense_19[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_6          │ (None, 248, 3563, │          0 │ input_layer_8[0]… │
│ (Multiply)          │ 1)                │            │ reshape_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_13 (Conv2D)  │ (None, 248, 3563, │        528 │ multiply_6[0][0]  │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 248, 3563, │         32 │ input_layer_8[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 248, 3563, │          0 │ conv2d_13[0][0],  │
│                     │ 16)               │            │ conv2d_14[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 248, 3563, │         64 │ add_3[0][0]       │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_10       │ (None, 248, 3563, │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling2d_8 │ (None, 248, 890,  │          0 │ activation_10[0]… │
│ (AveragePooling2D)  │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_12          │ (None, 248, 890,  │          0 │ average_pooling2… │
│ (Dropout)           │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 16)        │          0 │ dropout_12[0][0]  │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_6 (Reshape) │ (None, 1, 1, 16)  │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_20 (Dense)    │ (None, 1, 1, 31)  │        527 │ reshape_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_21 (Dense)    │ (None, 1, 1, 248) │      7,936 │ dense_20[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_7 (Reshape) │ (None, 248, 1, 1) │          0 │ dense_21[0][0]    │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 225,536,001 (860.35 MB)

 Trainable params: 225,535,905 (860.35 MB)

 Non-trainable params: 96 (384.00 B)


Starting Adapted AA-CascadeNet model training...
Epoch 1/100


In [1]:

model_save_path = '/content/drive/My Drive/DL_Project/eegnet_final_model.keras'
print(f"Saving model to: {model_save_path}")
eegnet_model.save(model_save_path)

print("Model saved successfully!")

Saving model to: /content/drive/My Drive/DL_Project/eegnet_final_model.keras


NameError: name 'eegnet_model' is not defined